In [1]:
import pandas as pd
import sources.raw as sr

In [95]:
raw_df = sr.load_raw_train_merged(nrows=500)

In [96]:
raw_df.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,shop_name,item_name,item_category_id,item_category_name
0,2013-01-02,0,59,22154,999.000000,1.0,"Ярославль ТЦ ""Альтаир""",ЯВЛЕНИЕ 2012 (BD),37,Кино - Blu-Ray
1,2013-01-03,0,25,2552,899.000000,1.0,"Москва ТРК ""Атриум""",DEEP PURPLE The House Of Blue Light LP,58,Музыка - Винил
2,2013-01-05,0,25,2552,899.000000,-1.0,"Москва ТРК ""Атриум""",DEEP PURPLE The House Of Blue Light LP,58,Музыка - Винил
3,2013-01-06,0,25,2554,1709.050049,1.0,"Москва ТРК ""Атриум""",DEEP PURPLE Who Do You Think We Are LP,58,Музыка - Винил
4,2013-01-15,0,25,2555,1099.000000,1.0,"Москва ТРК ""Атриум""",DEEP PURPLE 30 Very Best Of 2CD (Фирм.),56,Музыка - CD фирменного производства


In [103]:
def collect_features(df, observation_days = 30):
    start_ts = df['date'].min()
    finish_ts = df['date'].max()
    
    columns = ['ts', 'shop_id', 'item_category_id', 'item_id', 'sold_count_sum']
    data = {
        'ts' : [],
        'shop_id': [],
        'item_category_id': [],
        'item_id': [],
        'sold_count_sum': []
    }
    
#     print('{} - {}'.format(start_ts, finish_ts))
    
    while True:
        end_ts = start_ts + pd.Timedelta(days=observation_days)
#         print(start_ts, end_ts)

        observation_df = df[ (start_ts <= df['date']) & (df['date'] < finish_ts) ]
        keys_df = observation_df[['shop_id', 'item_category_id', 'item_id']].drop_duplicates()
#         print(keys_df.shape)
        for index, keys in keys_df.iterrows():
            shop_id = keys['shop_id']
            item_category_id = keys['item_category_id']
            item_id = keys['item_id']
            
#             print(shop_id, item_category_id, item_id)
            temp_df = observation_df[
                (observation_df['shop_id'] == shop_id) &
                (observation_df['item_category_id'] == item_category_id) &
                (observation_df['item_id'] == item_id)
            ]
        
            data['ts'].append(end_ts)
            data['shop_id'].append(shop_id)
            data['item_category_id'].append(item_category_id)
            data['item_id'].append(item_id)
            
            data['sold_count_sum'] = temp_df['item_cnt_day'].sum()
            
        start_ts += pd.Timedelta(days=1)
        if start_ts + pd.Timedelta(days=observation_days) > finish_ts:
            break
        
    result_df = pd.DataFrame(columns=columns, data=data)
    return result_df

In [106]:
df = collect_features(raw_df, 20)

In [107]:
df.head()

,ts,shop_id,item_category_id,item_id,sold_count_sum
0,2013-01-22,59,37,22154,1.0
1,2013-01-22,25,58,2552,1.0
2,2013-01-22,25,58,2554,1.0
3,2013-01-22,25,56,2555,1.0
4,2013-01-22,25,59,2564,1.0


In [108]:
df.tail()

,ts,shop_id,item_category_id,item_id,sold_count_sum
1622,2013-01-31,25,58,1292,1.0
1623,2013-01-31,25,55,1300,1.0
1624,2013-01-31,25,55,1249,1.0
1625,2013-01-31,25,55,1302,1.0
1626,2013-01-31,25,58,1205,1.0


In [109]:
df[df['sold_count_sum']>1]

,ts,shop_id,item_category_id,item_id,sold_count_sum
